In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from collections import OrderedDict
import cv2
import PIL
import keras
from keras.preprocessing.image import ImageDataGenerator
from keras.applications import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input
from keras import Model, layers
from keras.models import load_model, model_from_json
import tensorflow as tf

# Load the saved model
loaded_model = tf.keras.models.load_model("E:\PKLot/vehicle/model.h5")


#load function

#function for the testing image 
def test_image(image_path,model):
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # Convert from BGR to RGB

    # Resize the image to 224x224
    resized_image = cv2.resize(image, (224, 224))

    # Create an ImageDataGenerator for test with preprocessing function
    test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

    # Reshape the image to (1, height, width, channels)
    resized_image = np.expand_dims(resized_image, axis=0)

    # Generate batches of images
    batch_size = 1
    test_generator = test_datagen.flow(resized_image, batch_size=batch_size)
    predicted_labels = model.predict(test_generator)
    predicted_labels = np.argmax(predicted_labels, axis=1)
    #label maping
    label_mapping ={0:'Ambulance',1: 'Bicycle',2: 'Bus',3: 'Car',4: 'Limousine',5: 'Motorcycle',6: 'Segway',7: 'Tank',8: 'Taxi',9: 'Truck',10: 'Van'}
    predicted_string_labels = [label_mapping[label] for label in predicted_labels]
           
    return predicted_string_labels
   




#load window

import wx

class MyPanel(wx.Panel): 
    def __init__(self, parent): 
        super(MyPanel, self).__init__(parent)
        
        # Create a bitmap image
        self.bitmap = wx.StaticBitmap(self, pos=(10, 70))
        
        # Create a label for image type
        self.image_type_label = wx.StaticText(self, label="Vehicle Type : ", pos=(10, 10))
        self.label1 = wx.StaticText(self, label="None", pos=(100, 10))
        # Create a button
        button = wx.Button(self, label='Open Image', pos=(10, 30))
        button.Bind(wx.EVT_BUTTON, self.on_button_click)
        
    def on_button_click(self, event):
        wildcard = "Image files (*.jpg;*.png)|*.jpg;*.png|All files (*.*)|*.*"
        dlg = wx.FileDialog(self, message="Choose an image file", wildcard=wildcard,
                            style=wx.FD_OPEN | wx.FD_FILE_MUST_EXIST)
        if dlg.ShowModal() == wx.ID_OK:
            image_path = dlg.GetPath()
            image = wx.Image(image_path, wx.BITMAP_TYPE_ANY)
            image = image.Scale(224, 224, wx.IMAGE_QUALITY_HIGH)
            self.bitmap.SetBitmap(wx.Bitmap(image))
            
            # Get and display image type
             # Call the function to get predicted string labels
            predicted_string_labels = test_image(image_path, loaded_model)
            
            # Convert the list of predicted labels to a single string
            predicted_labels_string = ", ".join(predicted_string_labels)
            
            # Set the label1 text to the predicted string labels
            self.label1.SetLabel("Predicted Labels: " + predicted_labels_string)
            
        dlg.Destroy()

class Example(wx.Frame):
    def __init__(self, parent): 
        super(Example, self).__init__(parent,size=(400,400))  
        self.init_ui() 

    def init_ui(self):
        panel = MyPanel(self)
        self.SetTitle('Vehicle Type detection') 
        self.Centre() 
        self.Show(True)

if __name__ == '__main__':
    app = wx.App(False)
    Example(None) 
    app.MainLoop()
    

c:\ProgramData\anaconda3\envs\tf\lib\site-packages\h5py\__init__.py:36: UserWarning: h5py is running against HDF5 1.14.3 when it was built against 1.14.2, this may cause problems
  _warn(("h5py is running against HDF5 {0} when it was built against {1}, "


1/1 [==============================] - 1s 1s/step
